In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("/content/Train.csv")

In [3]:
df.Education_Level = np.sqrt(df.Education_Level)

In [4]:
df.Pay_Scale = df.Pay_Scale.fillna(6.0)

In [5]:
df.Work_Life_balance = df.Work_Life_balance.fillna(2.0)

In [6]:
df.Time_of_service = df.Time_of_service.fillna(6.0)

In [7]:
df.head()

,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
0,EID_23371,F,42.0,2.000000,Married,Franklin,IT,Conceptual,4.0,4,33,1,1,7.0,type2,3.0,4,0.7516,1.8688,2.0,4,5,3,0.1841
1,EID_18000,M,24.0,1.732051,Single,Springfield,Logistics,Analytical,5.0,4,36,0,3,6.0,type2,4.0,3,-0.9612,-0.4537,2.0,3,5,3,0.0670
2,EID_3891,F,58.0,1.732051,Married,Clinton,Quality,Conceptual,27.0,3,51,0,2,8.0,type2,1.0,4,-0.9612,-0.4537,3.0,3,8,3,0.0851
3,EID_17492,F,26.0,1.732051,Single,Lebanon,Human Resource Management,Behavioral,4.0,3,56,1,3,8.0,type2,1.0,3,-1.8176,-0.4537,NaN,3,7,3,0.0668
4,EID_22534,F,31.0,1.000000,Married,Springfield,Logistics,Conceptual,5.0,4,62,1,3,2.0,type3,3.0,1,0.7516,-0.4537,2.0,2,8,2,0.1827


In [8]:
df['Due'] = df['Time_of_service'] - df['Time_since_promotion']

In [9]:
# removing all irrelevant features

df_trial = df.drop(["VAR3", "VAR6", "VAR5", "VAR1", "Hometown"], axis = 1)

In [10]:
df_trial.VAR2 = df_trial.VAR2.fillna(0.7516)

In [11]:
df_trial.VAR4 = df_trial.VAR4.fillna(2.0)

In [12]:
df_trial.Age = df_trial.Age.fillna(25.0)

In [13]:
X = df_trial.drop(["Employee_ID", "Attrition_rate", 'Compensation_and_Benefits', 'Unit', "Education_Level", "Gender"], axis = 1)

In [14]:
X.head()

,Age,Relationship_Status,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Work_Life_balance,VAR2,VAR4,VAR7,Due
0,42.0,Married,Conceptual,4.0,4,33,1,1,7.0,3.0,0.7516,2.0,3,0.0
1,24.0,Single,Analytical,5.0,4,36,0,3,6.0,4.0,-0.9612,2.0,3,1.0
2,58.0,Married,Conceptual,27.0,3,51,0,2,8.0,1.0,-0.9612,3.0,3,24.0
3,26.0,Single,Behavioral,4.0,3,56,1,3,8.0,1.0,-1.8176,2.0,3,1.0
4,31.0,Married,Conceptual,5.0,4,62,1,3,2.0,3.0,0.7516,2.0,2,1.0


In [15]:
Y = df_trial.Attrition_rate

In [16]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)

In [17]:
from sklearn.compose import make_column_transformer
column_trans = make_column_transformer(
                (OneHotEncoder(), ['Decision_skill_possess', "Relationship_Status"]), 
                remainder = "passthrough")

In [18]:
X = column_trans.fit_transform(X)

In [19]:
X.shape

(7000, 18)

In [20]:
Y = np.array(Y)

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout

Using TensorFlow backend.


In [22]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(18, kernel_initializer='normal',input_dim = 18, activation='relu'))

# The Hidden Layers :
# NN_model.add(Dropout(0.3))
NN_model.add(Dense(12, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(6, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(12, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(6, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_squared_error', optimizer= "Adam", metrics=['mean_squared_error'])
NN_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 18)                342       
_________________________________________________________________
dense_2 (Dense)              (None, 12)                228       
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 78        
_________________________________________________________________
dense_4 (Dense)              (None, 12)                84        
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 78        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 7         
Total params: 817
Trainable params: 817
Non-trainable params: 0
________________________________________________________

In [23]:
NN_model.fit(X, Y, epochs = 5)

Epoch 1/5
7000/7000 [==============================] - 1s 72us/step - loss: 0.0399 - mean_squared_error: 0.0399
Epoch 2/5
7000/7000 [==============================] - 0s 45us/step - loss: 0.0349 - mean_squared_error: 0.0349
Epoch 3/5
7000/7000 [==============================] - 0s 43us/step - loss: 0.0346 - mean_squared_error: 0.0346
Epoch 4/5
7000/7000 [==============================] - 0s 46us/step - loss: 0.0346 - mean_squared_error: 0.0346
Epoch 5/5
7000/7000 [==============================] - 0s 47us/step - loss: 0.0345 - mean_squared_error: 0.0345


In [24]:
test = pd.read_csv("/content/Test.csv")

In [25]:
test = test.drop(["VAR3", "VAR6", "VAR5", "VAR1", "Hometown"], axis = 1)

In [26]:
test.Age = test.Age.fillna(27.0)

In [27]:
test.Work_Life_balance = test.Work_Life_balance.fillna(1.0)

In [28]:
test.Time_of_service = test.Time_of_service.fillna(4.0)

In [29]:
test.Pay_Scale = test.Pay_Scale.fillna(8.0)

In [30]:
test.VAR2 = test.VAR2.fillna(0.7516)

In [31]:
test.VAR4 = test.VAR4.fillna(2.0)

In [32]:
test['Due'] = test['Time_of_service'] - test['Time_since_promotion']

In [33]:
test_X = test.drop(['Employee_ID', 'Compensation_and_Benefits', "Unit", "Education_Level", "Gender"], axis = 1)

In [34]:
test_X = column_trans.fit_transform(test_X)

In [35]:
test_X.shape

(3000, 18)

In [36]:
y_pred = NN_model.predict(test_X)

In [37]:
y_pred.shape

(3000, 1)

In [38]:
np.savetxt("NNmodel_.csv", y_pred, delimiter = ",")